In [ ]:
from math import log2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import seaborn as sns
import os

drive.mount('/content/drive')
PATH = '/content/drive/MyDrive/Datasets/bc_categ.csv'

bc_df = pd.read_csv(PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
bc_df = pd.read_csv(PATH)

In [ ]:
#18, 32, 6751, 6783: Other ... 16, 40, 6800, 6805: Black ... 0, 1, 6814, 6815: White
bc_df = bc_df.iloc[[6000, 1, 2, 18, 32, 40, 6751, 6783, 6800, 685, 6814, 6815], :]
print(bc_df.to_latex(index=False,
                  float_format="{:.1f}".format,
))

\begin{tabular}{lllllllr}
\toprule
 Race & T Stage  & N Stage & 6th Stage & Grade & Estrogen Status & Progesterone Status &  Status \\
\midrule
White &       T1 &      N3 &      IIIC &     2 &        Positive &            Positive &       0 \\
White &       T2 &      N2 &      IIIA &     2 &        Positive &            Positive &       1 \\
White &       T3 &      N3 &      IIIC &     2 &        Positive &            Positive &       1 \\
Other &       T2 &      N1 &       IIB &     1 &        Positive &            Positive &       1 \\
Other &       T2 &      N3 &      IIIC &     2 &        Positive &            Positive &       1 \\
Black &       T2 &      N2 &      IIIA &     3 &        Positive &            Negative &       1 \\
Other &       T2 &      N2 &      IIIA &     2 &        Positive &            Positive &       0 \\
Other &       T4 &      N1 &      IIIB &     3 &        Negative &            Negative &       0 \\
Black &       T4 &      N2 &      IIIB &     2 &        

<ipython-input-50-d2a0c2c06bdd>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(bc_df.to_latex(index=False,


## Calculate the entropy of the entire dataset

In [ ]:
def calc_entropy(dataframe, target):
  target_vcounts = dataframe[target].value_counts()
  total_entropy = 0
  for value in target_vcounts.values:
    proportion = (value)/(dataframe.shape[0])
    total_entropy += proportion * log2(proportion)
  return -total_entropy

# For each unique value in target column we calculate the proportion (nunique)/(total samples)
# and multiply it by the log2(proportion). We then return the entropy multiplied by -1

In [ ]:
T2 = (5/6 * log2(5/6) + 1/6 * log2(1/6)) * -6/12
T2

0.32501121082417705

# Calculate the information gain for each column

In [ ]:
def information_gain(dataframe, column, target):
  val_count = dataframe[column].value_counts()
  # We get the value counts for the input column
  sum_subset = 0
  # we loop through val_count, extracting the index and the count
  for index, val in zip(val_count.index, val_count.values):
    # We get the proportion of nunique() / total samples
    proportion = val / dataframe.shape[0]
    # Extract the dataframe with the selected value from the column
    this_df = dataframe[dataframe[column] == index].copy()
    # Calculate the entropy for this dataset
    entropy = calc_entropy(this_df, target)
    # Add the product of proportion times entropy
    sum_subset += proportion * entropy
  # Subtract the sumset to the entropy to get the information gain
  info_gain = calc_entropy(dataframe, target) - sum_subset
  return info_gain

In [ ]:
information_gain(bc_df, 'T Stage ', 'Status')


0.44541483066220056

# Implement the ID3 algorithm

In [ ]:
def id3_algorithm(dataframe, target_column, attributes, tree=None):
    """
    @params:
    - dataframe: The dataset as a DataFrame
    - target_column: The target attribute as a string
    - attributes: The list of attributes for decision-making
    returns:
    - tree: The decision tree as a nested dictionary
    """
    # Get the unique values of the target attribute
    classes = dataframe[target_column].unique()
    # Case 1: If the dataset has all the same target value, return this value
    if len(classes) == 1:
        return classes[0]
    # Case 2: If the attribute list is empty, return the most frequent target value in the dataset
    if len(attributes) == 0:
        return dataframe[target_column].mode()[0]

    # Calculate the information gain for each attribute
    information_gains = {attribute: information_gain(dataframe, attribute, target_column) for attribute in attributes}
    # Choose the attribute with the highest information gain
    best_attribute = max(information_gains, key=information_gains.get)
    # Initialize the tree structure, if it's None
    if tree is None:
        tree = {}
        tree[best_attribute] = {}

    # Remove the best attribute from the attribute list
    attributes = [attr for attr in attributes if attr != best_attribute]
    # Grow the tree branch under the parent node
    for value in dataframe[best_attribute].unique():
        # Split the dataset based on the best attribute and its value
        subset = dataframe[dataframe[best_attribute] == value].drop([best_attribute], axis=1)
        # Call the ID3 algorithm recursively and add the subtree
        subtree = id3_algorithm(subset, target_column, attributes)

        # Add the new subtree to the parent tree
        tree[best_attribute][value] = subtree
    return tree


In [ ]:
id3_algorithm(bc_df, 'Status', bc_df.columns[:-1])

{'T Stage ': {'T1': 0,
  'T2': {'Race': {'White': 1,
    'Other': {'N Stage': {'N1': 1, 'N3': 1, 'N2': 0}},
    'Black': 1}},
  'T3': {'Grade': {'2': 1, '3': 0}},
  'T4': 0}}

## Measure the performance of the ID3 by implementing a function that makes the dictionary useful

In [ ]:
def classify_sample(tree, sample):

    # The stopping condition: if the tree is just a leaf node (0 or 1)
    if type(tree) is not dict:
        return tree

    # Get the attribute that the tree splits on
    attribute = list(tree.keys())[0]
    # Get the subtree corresponding to the attribute value in the sample
    if attribute in sample:  # Check if the attribute exists in the sample
        value = sample[attribute]
        subtree = tree[attribute].get(value)
        # If the value is not in the tree, we can't make a decision (could return a default value or None)
        if subtree is None:
            return None

        # Recursively call the classify_sample function on the subtree
        return classify_sample(subtree, sample)
    else:
        return None  # Attribute not in sample, can't make a decision

In [ ]:
# Define the evaluate_model_debug function with handling for default class
def evaluate_model_debug(tree, test_data, default_class):
    y_true = test_data['Status'].tolist()  # Actual labels
    y_pred = []  # Predicted labels

    # Iterate through each sample in the test data
    for i in range(len(test_data)):
        sample = test_data.iloc[i].to_dict()
        true_label = sample.pop('Status')  # Remove the target value to test the prediction
        prediction = classify_sample(tree, sample)

        # Use default class if prediction is None
        if prediction is None:
            prediction = default_class

        # Debugging information: print true and predicted labels
        print(f"Sample {i+1}: True label = {true_label}, Predicted label = {prediction}")

        y_pred.append(prediction)

    # Calculate accuracy and F1 score
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return accuracy, f1

# Determine the majority class in the training data
majority_class = train_data['Status'].mode()[0]

# Use a dummy tree for demonstration (replace this with your actual trained tree)
dummy_tree = {'T Stage ': {'T1': 0,
  'T2': {'Race': {'White': 1,
    'Other': {'N Stage': {'N1': 1, 'N3': 1, 'N2': 0}},
    'Black': 1}},
  'T3': {'Grade': {'2': 1, '3': 0}},
  'T4': 0}}

# Evaluate the model using a small subset of the test data and majority class as the default class
test_data_subset = test_data.sample(n=10, random_state=1)
accuracy, f1 = evaluate_model_debug(dummy_tree, test_data_subset, majority_class)
print(f"\nDebug Evaluation: Accuracy = {accuracy}, F1 Score = {f1}")



Sample 1: True label = 0, Predicted label = 1
Sample 2: True label = 1, Predicted label = 1
Sample 3: True label = 0, Predicted label = 0
Sample 4: True label = 0, Predicted label = 0
Sample 5: True label = 1, Predicted label = 1
Sample 6: True label = 0, Predicted label = 0
Sample 7: True label = 0, Predicted label = 1
Sample 8: True label = 1, Predicted label = 0
Sample 9: True label = 1, Predicted label = 1
Sample 10: True label = 0, Predicted label = 1

Debug Evaluation: Accuracy = 0.6, F1 Score = 0.6


# Use this to generate a string to diagram mermaid can plot a decision tree

In [ ]:
def dict_to_mermaid(d, parent=None, mermaid_str=[]):
    """
    Convert a nested dictionary to a Mermaid graph definition string.

    Parameters:
    d (dict): The nested dictionary to convert.
    parent (str): The parent node in the Mermaid graph.
    mermaid_str (list): The list to store Mermaid graph elements.

    Returns:
    str: The Mermaid graph definition string.
    """
    for key, value in d.items():
        node = f"{key}"
        if parent:
            mermaid_str.append(f"{parent} --> {node}")
        if isinstance(value, dict):
            dict_to_mermaid(value, node, mermaid_str)
        else:
            leaf_node = f"{node}_{value}"
            mermaid_str.append(f"{node} --> {leaf_node}")
            mermaid_str.append(f"class {leaf_node} end;")
    return "graph TD;\n" + ";\n".join(mermaid_str) + ";"

# Given nested dictionary
nested_dict = {
    'Race': {'White': {'6th Stage': {'IIA': {'Progesterone Status': {'Positive': {'Grade': {'3': {'T Stage ': {'T1': {'N Stage': {'N1': {'Estrogen Status': {'Positive': 0}}}}}},
        '2': 1,
        '1': 1}},
      'Negative': 0}},
    'IIIA': 1,
    'IIIC': {'T Stage ': {'T3': {'Grade': {'2': 1, '1': 0, '3': 0}}, 'T4': 1}},
    'IIB': {'Grade': {'3': {'Estrogen Status': {'Positive': 1, 'Negative': 0}},
      '2': 0}},
    'IIIB': 0}},
  'Black': 0}
}

# Generate Mermaid graph definition string
mermaid_str = dict_to_mermaid(nested_dict)
print(mermaid_str)


graph TD;
Race --> White;
White --> 6th Stage;
6th Stage --> IIA;
IIA --> Progesterone Status;
Progesterone Status --> Positive;
Positive --> Grade;
Grade --> 3;
3 --> T Stage ;
T Stage  --> T1;
T1 --> N Stage;
N Stage --> N1;
N1 --> Estrogen Status;
Estrogen Status --> Positive;
Positive --> Positive_0;
class Positive_0 end;;
Grade --> 2;
2 --> 2_1;
class 2_1 end;;
Grade --> 1;
1 --> 1_1;
class 1_1 end;;
Progesterone Status --> Negative;
Negative --> Negative_0;
class Negative_0 end;;
6th Stage --> IIIA;
IIIA --> IIIA_1;
class IIIA_1 end;;
6th Stage --> IIIC;
IIIC --> T Stage ;
T Stage  --> T3;
T3 --> Grade;
Grade --> 2;
2 --> 2_1;
class 2_1 end;;
Grade --> 1;
1 --> 1_0;
class 1_0 end;;
Grade --> 3;
3 --> 3_0;
class 3_0 end;;
T Stage  --> T4;
T4 --> T4_1;
class T4_1 end;;
6th Stage --> IIB;
IIB --> Grade;
Grade --> 3;
3 --> Estrogen Status;
Estrogen Status --> Positive;
Positive --> Positive_1;
class Positive_1 end;;
Estrogen Status --> Negative;
Negative --> Negative_0;
class Negativ